In [1]:
import os
import numpy
import scipy.io
import pandas as pd
import matplotlib.pyplot as plt
import keras

/dvlscratch/SHI/users/hofman/ML/venv3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using Theano backend.


In [2]:
STA = 'STKA'

In [3]:
#load data from oracle do pandas
#connect to udb
with open(os.path.join('/','home','hofman','.dbp.txt'), 'r') as f: password_old = f.read().strip()
import cx_Oracle
connection = cx_Oracle.connect('hofman', password_old, 'udb')

In [4]:
from datetime import datetime as dt
ts_2017 = dt(2017,1,1,0,0,0).timestamp()
ts_2018 = dt(2018,1,1,0,0,0).timestamp()
ts_2019 = dt(2019,1,1,0,0,0).timestamp()

In [27]:
query = f"select * from EVAL_QANDEF_STKA"   ## where sta='{STA}'"
df_all_ = pd.read_sql(query, con=connection)

In [28]:
df_all_.shape

(153816, 12)

In [29]:
ind = [{False: True, True:False}[x] for x in numpy.isnan(df_all_['NCLS_PHASE_ODB'])]

In [30]:
df_all = df_all_[ind]

In [31]:
df_all.head()

,ARID_ODB,TIME_ODB,AZI_ODB,IPHASE_QANDEF,IPHASE_ODB,ASSOC_PHASE,CLASS_IPHASE_QANDEF,CLASS_IPHASE_ODB,CLASS_PHASE_ODB,NCLS_IPHASE_QANDEF,NCLS_IPHASE_ODB,NCLS_PHASE_ODB
0,128815337,1.514765e+09,134.064260,N,N,S,N,N,T,0,0,1
1,128815338,1.514765e+09,108.852400,N,P,None,N,T,N,0,1,0
2,128815542,1.514766e+09,320.088700,N,Sx,None,N,regS,N,0,3,0
3,128815688,1.514766e+09,343.943600,P,N,P,T,N,T,1,0,1
4,128815962,1.514768e+09,16.134881,tx,tx,None,T,T,N,1,1,0


In [32]:
import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    NCLS_IPHASE_QANDEF = df_all['NCLS_IPHASE_QANDEF'].as_matrix()
    NCLS_IPHASE_ODB = df_all['NCLS_IPHASE_ODB'].as_matrix()
    NCLS_PHASE_ODB = numpy.array(df_all['NCLS_PHASE_ODB'].as_matrix(), dtype=numpy.int)

In [33]:
NCLS_PHASE_ODB.min(), NCLS_PHASE_ODB.max()

(0, 3)

In [34]:
df_all[numpy.isnan(df_all['NCLS_PHASE_ODB'])]


,ARID_ODB,TIME_ODB,AZI_ODB,IPHASE_QANDEF,IPHASE_ODB,ASSOC_PHASE,CLASS_IPHASE_QANDEF,CLASS_IPHASE_ODB,CLASS_PHASE_ODB,NCLS_IPHASE_QANDEF,NCLS_IPHASE_ODB,NCLS_PHASE_ODB


In [35]:
from sklearn.metrics import confusion_matrix

In [46]:
def confmat(x,y):
    C = confusion_matrix(x,y)
    diagsum = numpy.diag(C).sum()
    accuracy = diagsum/C.sum()
    print(C)
    print(C.sum(axis=0), C.sum()) 
    AACC = (numpy.diag(C)[1:].sum())/(C[:,1:].sum())*100
    ACC = (accuracy*100)
    N = (C[0,1:].sum()/C[:,1:].sum()*100)
    print('Overal accuracy: %3.1f%%' % ACC)
    print('Assoc accuracy: %3.1f%%' % AACC)
    print('N phase rate: %3.1f%%' % N)
    return ACC, AACC, N

In [47]:
#confusion_matrix(y_true, y_pred)
print('OLD:')
ACC1, AACC1, N1 = confmat(NCLS_IPHASE_ODB, NCLS_PHASE_ODB)

print('NEW')
ACC2, AACC2, N2 = confmat(NCLS_IPHASE_QANDEF, NCLS_PHASE_ODB)

print('Difference:')
print("%3.1f%%" % (ACC2-ACC1), " %3.1f%%" % (AACC2-AACC1), " %3.1f%%" % (N2-N1))

OLD:
[[86809   675     3    19]
 [39520  6304     5     3]
 [11789  1311    44    17]
 [ 7256    29     0    32]]
[145374   8319     52     71] 153816
Overal accuracy: 60.6%
Assoc accuracy: 75.6%
N phase rate: 8.3%
NEW
[[93523   275     3    45]
 [39775  6806     7     9]
 [ 9841  1209    42    10]
 [ 2235    29     0     7]]
[145374   8319     52     71] 153816
Overal accuracy: 65.3%
Assoc accuracy: 81.2%
N phase rate: 3.8%
Difference:
4.7%  5.6%  -4.4%


In [23]:
45+9+10+7
19+3+17+32
print(86809+39520+11789+7256)
print(92719+39754+10057+2884)


145374
145414


partial results, retreined just n-tps:

OLD:
[[86809   675     3    19]
 [39520  6304     5     3]
 [11789  1311    44    17]
 [ 7256    29     0    32]]
[145374   8319     52     71] 153816
Assoc accuracy: 75.57%
Overal accuracy: 60.58%
N phase rate: 8.26%
NEW
[[92719   306     3    41]
 [39754  6757     7     8]
 [10057  1226    42    11]
 [ 2844    30     0    11]]
[145374   8319     52     71] 153816
Assoc accuracy: 80.67%
Overal accuracy: 64.71%
N phase rate: 4.15%

=================================================

OLD:
[[86809   675     3    19]
 [39520  6304     5     3]
 [11789  1311    44    17]
 [ 7256    29     0    32]]
Assoc accuracy: 75.57%
Overal accuracy: 60.58%
N phase rate: 8.26%
NEW
[[92719   306     3    41]
 [39754  6757     7     8]
 [10057  1226    42    11]
 [ 2844    30     0    11]]
Assoc accuracy: 80.67%
Overal accuracy: 64.71%
N phase rate: 4.15%


OLD:
[[86809   675     3    19]
 [39520  6304     5     3]
 [11789  1311    44    17]
 [ 7256    29     0    32]]
Assoc accuracy: 75.57%
Overal accuracy: 60.58%
N phase rate: 8.26%
NEW
[[93523   275     3    45]
 [21446  5978     3     3]
 [ 4883  1309    26     4]
 [25522   757    20    19]]
Assoc accuracy: 71.35%
Overal accuracy: 64.72%
N phase rate: 3.83%


model_NTPS = load_model('h5/%s_NTPS20.h5' % STA)
model_STP = load_model('h5/%s_TPS3.h5' % STA)
model_PT = load_model('h5/%s_TP3.h5' % STA)
OLD:
[[86809   675     3    19]
 [39520  6304     5     3]
 [11789  1311    44    17]
 [ 7256    29     0    32]]
Assoc accuracy: 75.57%
Overal accuracy: 60.58%
N phase rate: 8.26%
NEW
[[89283   657     1    10]
 [12462  5578     4     0]
 [10129  1726    41    10]
 [33500   358     6    51]]
Assoc accuracy: 67.16%
Overal accuracy: 61.73%
N phase rate: 7.91%


OLD:
[[86809   675     3    19]
 [39520  6304     5     3]
 [11789  1311    44    17]
 [ 7256    29     0    32]]
Assoc accuracy: 75.57%
Overal accuracy: 60.58%
N phase rate: 8.26%
NEW
[[89662   701     1     8]
 [11293  5535     5     1]
 [ 9170  1724    40    10]
 [35249   359     6    52]]
Assoc accuracy: 66.65%
Overal accuracy: 61.95%
N phase rate: 8.41%


OLD:
[[86809   675     3    19]
 [39520  6304     5     3]
 [11789  1311    44    17]
 [ 7256    29     0    32]]
Assoc accuracy: 75.57%
Overal accuracy: 60.58%
N phase rate: 8.26%
NEW
[[93523   275     3    45]
 [21446  5978     3     3]
 [ 4883  1309    26     4]
 [25522   757    20    19]]
Assoc accuracy: 71.35%
Overal accuracy: 64.72%
N phase rate: 3.83%


In [35]:
a = {x+1:('a','b','c')[x] for x in range(3)}

In [36]:
a


{1: 'a', 2: 'b', 3: 'c'}